In [ ]:
from glob import glob
import os
import numpy as np
import cv2
from sklearn.metrics import roc_curve, precision_recall_curve, auc
from keras.datasets import mnist, fashion_mnist, cifar100, cifar10
from keras.backend import cast_to_floatx
import random

In [ ]:
from keras.models import Model
from keras.layers import Input, Add, Activation, Dropout, Flatten, Dense
from keras.layers.convolutional import Convolution2D, MaxPooling2D, AveragePooling2D
#from keras.layers.normalization import BatchNormalization
from keras.layers.normalization.batch_normalization_v1 import BatchNormalization
from keras.regularizers import l2
from keras import backend as K

random.seed(0)
weight_decay = 0.000005

In [ ]:
import abc
import itertools
import numpy as np
from keras.preprocessing.image import apply_affine_transform

## **Load and Preprocessing the data**

In [ ]:
def resize_and_crop_image(input_file, output_side_length, greyscale=False):
    img = cv2.imread(input_file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB if not greyscale else cv2.COLOR_BGR2GRAY)
    height, width = img.shape[:2]
    new_height = output_side_length
    new_width = output_side_length
    if height > width:
        new_height = int(output_side_length * height / width)
    else:
        new_width = int(output_side_length * width / height)
    resized_img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_AREA)
    height_offset = (new_height - output_side_length) // 2
    width_offset = (new_width - output_side_length) // 2
    cropped_img = resized_img[height_offset:height_offset + output_side_length,
                              width_offset:width_offset + output_side_length]
    assert cropped_img.shape[:2] == (output_side_length, output_side_length)
    return cropped_img

In [ ]:
def normalize_minus1_1(data):
    return 2*(data/255.) - 1

In [ ]:
def get_channels_axis():
  import keras
  idf = keras.backend.image_data_format()
  if idf == 'channels_first':
      return 1
  assert idf == 'channels_last'
  return 3

In [ ]:
def load_cifar10():
    (X_train, y_train), (X_test, y_test) = cifar10.load_data()
    X_train = normalize_minus1_1(cast_to_floatx(X_train))
    X_test = normalize_minus1_1(cast_to_floatx(X_test))
    return (X_train, y_train), (X_test, y_test)

## **Saving the results**

In [ ]:
def save_roc_pr_curve_data(scores, labels):
    scores = scores.flatten()
    labels = labels.flatten()

    scores_pos = scores[labels == 1]
    scores_neg = scores[labels != 1]

    truth = np.concatenate((np.zeros_like(scores_neg), np.ones_like(scores_pos)))
    preds = np.concatenate((scores_neg, scores_pos))
    fpr, tpr, roc_thresholds = roc_curve(truth, preds)
    roc_auc = auc(fpr, tpr)
    print("roc_auc",roc_auc)

    # pr curve where "normal" is the positive class
    precision_norm, recall_norm, pr_thresholds_norm = precision_recall_curve(truth, preds)
    pr_auc_norm = auc(recall_norm, precision_norm)
    print("pr_auc_norm where normal is the positive class",pr_auc_norm)

    # pr curve where "anomaly" is the positive class
    precision_anom, recall_anom, pr_thresholds_anom = precision_recall_curve(truth, -preds, pos_label=0)
    pr_auc_anom = auc(recall_anom, precision_anom)
    print("pr_auc_norm where anomaly is the positive class",pr_auc_anom)

In [ ]:
def get_class_name_from_index(index, dataset_name):
    ind_to_name = {
        'cifar10': ('airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'),
    }

    return ind_to_name[dataset_name][index]

## **Transformations**

In [ ]:
class AffineTransformation(object):
    def __init__(self, flip, tx, ty, k_90_rotate):
        self.flip = flip
        self.tx = tx
        self.ty = ty
        self.k_90_rotate = k_90_rotate

    def __call__(self, x):
        res_x = x
        if self.flip:
            res_x = np.fliplr(res_x)
        if self.tx != 0 or self.ty != 0:
            res_x = apply_affine_transform(res_x, tx=self.tx, ty=self.ty,row_axis=0,
    col_axis=1, channel_axis=2, fill_mode='reflect')
        if self.k_90_rotate != 0:
            res_x = np.rot90(res_x, self.k_90_rotate)

        return res_x

In [ ]:
class AbstractTransformer(abc.ABC):
    def __init__(self):
        self._transformation_list = None
        self._create_transformation_list()

    @property
    def n_transforms(self):
        return len(self._transformation_list)

    @abc.abstractmethod
    def _create_transformation_list(self):
        return

    def transform_batch(self, x_batch, t_inds):
        assert len(x_batch) == len(t_inds)

        transformed_batch = x_batch.copy()
        for i, t_ind in enumerate(t_inds):
            transformed_batch[i] = self._transformation_list[t_ind](transformed_batch[i])
        return transformed_batch

In [ ]:
class Transformer(AbstractTransformer):
    def __init__(self, translation_x=8, translation_y=8):
        self.max_tx = translation_x
        self.max_ty = translation_y
        super().__init__()

    def _create_transformation_list(self):
        transformation_list = []
        for is_flip, tx, ty, k_rotate in itertools.product((False, True),
                                                           (0, -self.max_tx, self.max_tx),
                                                           (0, -self.max_ty, self.max_ty),
                                                           range(4)):
            transformation = AffineTransformation(is_flip, tx, ty, k_rotate)
            transformation_list.append(transformation)

        self._transformation_list = transformation_list

In [ ]:
class SimpleTransformer(AbstractTransformer):
    def _create_transformation_list(self):
        transformation_list = []
        for is_flip, k_rotate in itertools.product((False, True),
                                                    range(4)):
            transformation = AffineTransformation(is_flip, 0, 0, k_rotate)
            transformation_list.append(transformation)

        self._transformation_list = transformation_list

## **The Model: Wide Residual Network**

In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D, Activation, BatchNormalization, Flatten, Dense, Conv2DTranspose, Reshape


def conv_encoder(input_side=32, n_channels=3, representation_dim=256, representation_activation='tanh',
                 intermediate_activation='relu'):
    nf = 64
    input_shape = (n_channels, input_side, input_side) if get_channels_axis() == 1 else (input_side, input_side,
                                                                                         n_channels)

    x_in = Input(shape=input_shape)
    enc = x_in

    # downsample x0.5
    enc = Conv2D(nf, kernel_size=(3, 3), strides=(2, 2), padding='same')(enc)
    enc = BatchNormalization(axis=get_channels_axis())(enc)
    enc = Activation(intermediate_activation)(enc)

    # downsample x0.5
    enc = Conv2D(nf * 2, kernel_size=(3, 3), strides=(2, 2), padding='same')(enc)
    enc = BatchNormalization(axis=get_channels_axis())(enc)
    enc = Activation(intermediate_activation)(enc)

    # downsample x0.5
    enc = Conv2D(nf * 4, kernel_size=(3, 3), strides=(2, 2), padding='same')(enc)
    enc = BatchNormalization(axis=get_channels_axis())(enc)
    enc = Activation(intermediate_activation)(enc)

    if input_side == 64:
        # downsample x0.5
        enc = Conv2D(nf * 8, kernel_size=(3, 3), strides=(2, 2), padding='same')(enc)
        enc = BatchNormalization(axis=get_channels_axis())(enc)
        enc = Activation(intermediate_activation)(enc)

    enc = Flatten()(enc)
    rep = Dense(representation_dim, activation=representation_activation)(enc)

    return Model(x_in, rep)


def conv_decoder(output_side=32, n_channels=3, representation_dim=256, activation='relu'):
    nf = 64

    rep_in = Input(shape=(representation_dim,))

    g = Dense(nf * 4 * 4 * 4)(rep_in)
    g = BatchNormalization(axis=-1)(g)
    g = Activation(activation)(g)

    conv_shape = (nf * 4, 4, 4) if get_channels_axis() == 1 else (4, 4, nf * 4)
    g = Reshape(conv_shape)(g)

    # upsample x2
    g = Conv2DTranspose(nf * 2, kernel_size=(3, 3), strides=(2, 2), padding='same')(g)
    g = BatchNormalization(axis=get_channels_axis())(g)
    g = Activation(activation)(g)

    # upsample x2
    g = Conv2DTranspose(nf, kernel_size=(3, 3), strides=(2, 2), padding='same')(g)
    g = BatchNormalization(axis=get_channels_axis())(g)
    g = Activation(activation)(g)

    if output_side == 64:
        # upsample x2
        g = Conv2DTranspose(nf, kernel_size=(3, 3), strides=(2, 2), padding='same')(g)
        g = BatchNormalization(axis=get_channels_axis())(g)
        g = Activation(activation)(g)

    # upsample x2
    g = Conv2DTranspose(n_channels, kernel_size=(3, 3), strides=(2, 2), padding='same')(g)
    g = Activation('tanh')(g)

    return Model(rep_in, g)

## **Experiments**

In [ ]:
import os
import csv
from collections import defaultdict
from glob import glob
from datetime import datetime
from multiprocessing import Manager, freeze_support, Process
import numpy as np
import scipy.stats
from scipy.special import psi, polygamma
from sklearn.metrics import roc_auc_score
from sklearn.svm import OneClassSVM
from sklearn.model_selection import ParameterGrid
from joblib import Parallel, delayed
from keras.models import Model, Sequential
from tensorflow.keras.layers import Input
from keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
from glob import glob
import os
import numpy as np
import cv2
from sklearn.metrics import roc_curve, precision_recall_curve, auc
from keras.datasets import mnist, fashion_mnist, cifar100, cifar10
from keras.backend import cast_to_floatx
from tqdm.notebook import tqdm

In [ ]:
def _train_ocsvm_and_score(params, xtrain, test_labels, xtest):
    return roc_auc_score(test_labels, OneClassSVM(**params).fit(xtrain).decision_function(xtest))
def _cae_ocsvm_experiment(dataset_load_fn, dataset_name, single_class_ind, gpu_q):
    print('Class:', single_class_ind)
    gpu_to_use = gpu_q.get()
    os.environ["CUDA_VISIBLE_DEVICES"] = gpu_to_use

    (x_train, y_train), (x_test, y_test) = dataset_load_fn()
    x_train = x_train[:TRAIN_SIZE]
    y_train =  y_train[:TRAIN_SIZE]
    x_test =  x_test[:VAL_SIZE]
    y_test =  y_test[:VAL_SIZE]

    n_channels = x_train.shape[get_channels_axis()]
    input_side = x_train.shape[2]  # channel side will always be at shape[2]
    enc = conv_encoder(input_side, n_channels)
    dec = conv_decoder(input_side, n_channels)
    x_in = Input(shape=x_train.shape[1:])
    x_rec = dec(enc(x_in))
    cae = Model(x_in, x_rec)
    cae.compile('adam', 'mse')

    x_train_task = x_train[y_train.flatten() == single_class_ind]
    x_test_task = x_test[y_test.flatten() == single_class_ind]  # This is just for visual monitoring
    cae.fit(x=x_train_task, y=x_train_task, batch_size=128, epochs=10, validation_data=(x_test_task, x_test_task))

    x_train_task_rep = enc.predict(x_train_task, batch_size=128)

    x_test_rep = enc.predict(x_test, batch_size=128)
    pg = ParameterGrid({'nu': np.linspace(0.1, 0.9, num=9),
                        'gamma': np.logspace(-7, 2, num=10, base=2)})

    results = Parallel(n_jobs=6)(
        delayed(_train_ocsvm_and_score)(d, x_train_task_rep, y_test.flatten() == single_class_ind, x_test_rep)
        for d in pg)

    best_params, best_auc_score = max(zip(pg, results), key=lambda t: t[-1])
    best_ocsvm = OneClassSVM(**best_params).fit(x_train_task_rep)
    scores = best_ocsvm.decision_function(x_test_rep)
    labels = y_test.flatten() == single_class_ind

    save_roc_pr_curve_data(scores, labels)

    gpu_q.put(gpu_to_use)

In [ ]:
def run_experiments(load_dataset_fn, dataset_name, q, n_classes):
    print("START _transformations_experiment ")
    # CAE OC-SVM
    n_runs = 3
    for i in range(n_runs):
        print('Run Number:',i+1)
        processes = [Process(target=_cae_ocsvm_experiment,
                         args=(load_dataset_fn, dataset_name, c, q)) for c in range(n_classes)]
        for p in processes:
            p.start()
            p.join()
    print("END _transformations_experiment ")

In [ ]:
RESULTS_DIR = ''
TRAIN_SIZE = 10000
VAL_SIZE = 1000
BATCH_SIZE = 64

In [ ]:
if __name__ == '__main__':
    freeze_support()
    N_GPUS = 1
    man = Manager()
    q = man.Queue(N_GPUS)
    for g in range(N_GPUS):
        q.put(str(g))

    experiments_list = [ 
        (load_cifar10, 'cifar10', 10),
    ]

    for data_load_fn, dataset_name, n_classes in experiments_list:
        run_experiments(data_load_fn, dataset_name, q, n_classes)

START _transformations_experiment 
Run Number: 1
Class: 0
Epoch 1/10
8/8 [==============================] - ETA: 9s - loss: 0.391 - ETA: 1s - loss: 0.360 - ETA: 0s - loss: 0.341 - ETA: 0s - loss: 0.321 - ETA: 0s - loss: 0.303 - ETA: 0s - loss: 0.286 - ETA: 0s - loss: 0.269 - ETA: 0s - loss: 0.259 - 3s 239ms/step - loss: 0.2597 - val_loss: 0.2727
Epoch 2/10
8/8 [==============================] - ETA: 0s - loss: 0.158 - ETA: 0s - loss: 0.152 - ETA: 0s - loss: 0.148 - ETA: 0s - loss: 0.140 - ETA: 0s - loss: 0.139 - ETA: 0s - loss: 0.135 - ETA: 0s - loss: 0.131 - ETA: 0s - loss: 0.129 - 1s 142ms/step - loss: 0.1295 - val_loss: 0.2367
Epoch 3/10
8/8 [==============================] - ETA: 0s - loss: 0.110 - ETA: 0s - loss: 0.102 - ETA: 0s - loss: 0.101 - ETA: 0s - loss: 0.098 - ETA: 0s - loss: 0.098 - ETA: 0s - loss: 0.097 - ETA: 0s - loss: 0.096 - ETA: 0s - loss: 0.096 - 1s 161ms/step - loss: 0.0965 - val_loss: 0.2077
Epoch 4/10
8/8 [==============================] - ETA: 0s - loss: 0.090 

9/9 [==============================] - ETA: 1s - loss: 0.063 - ETA: 1s - loss: 0.064 - ETA: 1s - loss: 0.063 - ETA: 0s - loss: 0.062 - ETA: 0s - loss: 0.063 - ETA: 0s - loss: 0.062 - ETA: 0s - loss: 0.062 - ETA: 0s - loss: 0.061 - ETA: 0s - loss: 0.062 - 2s 170ms/step - loss: 0.0624 - val_loss: 0.1624
Epoch 8/10
9/9 [==============================] - ETA: 0s - loss: 0.064 - ETA: 1s - loss: 0.064 - ETA: 1s - loss: 0.063 - ETA: 0s - loss: 0.062 - ETA: 0s - loss: 0.061 - ETA: 0s - loss: 0.061 - ETA: 0s - loss: 0.060 - ETA: 0s - loss: 0.059 - 1s 151ms/step - loss: 0.0598 - val_loss: 0.1562
Epoch 9/10
9/9 [==============================] - ETA: 1s - loss: 0.055 - ETA: 1s - loss: 0.056 - ETA: 0s - loss: 0.056 - ETA: 0s - loss: 0.056 - ETA: 0s - loss: 0.057 - ETA: 0s - loss: 0.057 - ETA: 0s - loss: 0.057 - ETA: 0s - loss: 0.056 - 1s 157ms/step - loss: 0.0573 - val_loss: 0.1543
Epoch 10/10
9/9 [==============================] - ETA: 1s - loss: 0.059 - ETA: 1s - loss: 0.058 - ETA: 0s - loss: 0.

8/8 [==============================] - ETA: 0s - loss: 0.123 - ETA: 1s - loss: 0.119 - ETA: 0s - loss: 0.119 - ETA: 0s - loss: 0.116 - ETA: 0s - loss: 0.115 - ETA: 0s - loss: 0.114 - ETA: 0s - loss: 0.112 - ETA: 0s - loss: 0.111 - 1s 157ms/step - loss: 0.1119 - val_loss: 0.1985
Epoch 4/10
8/8 [==============================] - ETA: 1s - loss: 0.099 - ETA: 0s - loss: 0.100 - ETA: 0s - loss: 0.099 - ETA: 0s - loss: 0.098 - ETA: 0s - loss: 0.098 - ETA: 0s - loss: 0.097 - ETA: 0s - loss: 0.096 - ETA: 0s - loss: 0.096 - 1s 167ms/step - loss: 0.0962 - val_loss: 0.1837
Epoch 5/10
8/8 [==============================] - ETA: 0s - loss: 0.087 - ETA: 1s - loss: 0.087 - ETA: 0s - loss: 0.086 - ETA: 0s - loss: 0.087 - ETA: 0s - loss: 0.087 - ETA: 0s - loss: 0.087 - ETA: 0s - loss: 0.086 - ETA: 0s - loss: 0.086 - 2s 198ms/step - loss: 0.0869 - val_loss: 0.1824
Epoch 6/10
8/8 [==============================] - ETA: 1s - loss: 0.081 - ETA: 1s - loss: 0.081 - ETA: 0s - loss: 0.080 - ETA: 0s - loss: 0.0

8/8 [==============================] - ETA: 1s - loss: 0.067 - ETA: 1s - loss: 0.066 - ETA: 0s - loss: 0.066 - ETA: 0s - loss: 0.066 - ETA: 0s - loss: 0.066 - ETA: 0s - loss: 0.066 - ETA: 0s - loss: 0.066 - ETA: 0s - loss: 0.067 - 2s 197ms/step - loss: 0.0670 - val_loss: 0.1917
Epoch 10/10
8/8 [==============================] - ETA: 1s - loss: 0.064 - ETA: 0s - loss: 0.066 - ETA: 0s - loss: 0.064 - ETA: 0s - loss: 0.064 - ETA: 0s - loss: 0.064 - ETA: 0s - loss: 0.064 - ETA: 0s - loss: 0.064 - ETA: 0s - loss: 0.064 - 2s 219ms/step - loss: 0.0647 - val_loss: 0.1891
roc_auc 0.5425346084981877
pr_auc_norm where normal is the positive class 0.11293405594719379
pr_auc_norm where anomaly is the positive class 0.9113179175001034
Class: 8
Epoch 1/10
9/9 [==============================] - ETA: 12s - loss: 0.38 - ETA: 0s - loss: 0.3601 - ETA: 1s - loss: 0.338 - ETA: 0s - loss: 0.315 - ETA: 0s - loss: 0.296 - ETA: 0s - loss: 0.281 - ETA: 0s - loss: 0.267 - ETA: 0s - loss: 0.254 - ETA: 0s - loss: 0

Epoch 5/10
8/8 [==============================] - ETA: 0s - loss: 0.072 - ETA: 0s - loss: 0.074 - ETA: 0s - loss: 0.075 - ETA: 0s - loss: 0.076 - ETA: 0s - loss: 0.075 - ETA: 0s - loss: 0.075 - ETA: 0s - loss: 0.075 - ETA: 0s - loss: 0.075 - 1s 166ms/step - loss: 0.0754 - val_loss: 0.1868
Epoch 6/10
8/8 [==============================] - ETA: 0s - loss: 0.068 - ETA: 0s - loss: 0.068 - ETA: 0s - loss: 0.068 - ETA: 0s - loss: 0.068 - ETA: 0s - loss: 0.068 - ETA: 0s - loss: 0.069 - ETA: 0s - loss: 0.068 - ETA: 0s - loss: 0.068 - 1s 179ms/step - loss: 0.0685 - val_loss: 0.1980
Epoch 7/10
8/8 [==============================] - ETA: 1s - loss: 0.061 - ETA: 1s - loss: 0.062 - ETA: 0s - loss: 0.063 - ETA: 0s - loss: 0.063 - ETA: 0s - loss: 0.063 - ETA: 0s - loss: 0.063 - ETA: 0s - loss: 0.063 - ETA: 0s - loss: 0.063 - 2s 204ms/step - loss: 0.0636 - val_loss: 0.1976
Epoch 8/10
8/8 [==============================] - ETA: 1s - loss: 0.062 - ETA: 1s - loss: 0.059 - ETA: 0s - loss: 0.059 - ETA: 0s 

...

Epoch 3/10
9/9 [==============================] - ETA: 1s - loss: 0.104 - ETA: 1s - loss: 0.105 - ETA: 1s - loss: 0.106 - ETA: 0s - loss: 0.103 - ETA: 0s - loss: 0.103 - ETA: 0s - loss: 0.103 - ETA: 0s - loss: 0.100 - ETA: 0s - loss: 0.099 - 1s 157ms/step - loss: 0.0993 - val_loss: 0.1817
Epoch 4/10
9/9 [==============================] - ETA: 0s - loss: 0.083 - ETA: 1s - loss: 0.086 - ETA: 0s - loss: 0.085 - ETA: 0s - loss: 0.086 - ETA: 0s - loss: 0.086 - ETA: 0s - loss: 0.085 - ETA: 0s - loss: 0.085 - ETA: 0s - loss: 0.085 - ETA: 0s - loss: 0.085 - 1s 145ms/step - loss: 0.0852 - val_loss: 0.1739
Epoch 5/10
9/9 [==============================] - ETA: 0s - loss: 0.085 - ETA: 1s - loss: 0.081 - ETA: 0s - loss: 0.079 - ETA: 0s - loss: 0.077 - ETA: 0s - loss: 0.076 - ETA: 0s - loss: 0.075 - ETA: 0s - loss: 0.075 - ETA: 0s - loss: 0.076 - ETA: 0s - loss: 0.076 - 1s 167ms/step - loss: 0.0761 - val_loss: 0.1636
Epoch 6/10
9/9 [==============================] - ETA: 1s - loss: 0.077 - ETA: 1s 

8/8 [==============================] - ETA: 0s - loss: 0.059 - ETA: 0s - loss: 0.059 - ETA: 0s - loss: 0.059 - ETA: 0s - loss: 0.060 - ETA: 0s - loss: 0.059 - ETA: 0s - loss: 0.059 - ETA: 0s - loss: 0.059 - ETA: 0s - loss: 0.059 - 1s 166ms/step - loss: 0.0592 - val_loss: 0.1832
Epoch 10/10
8/8 [==============================] - ETA: 0s - loss: 0.058 - ETA: 0s - loss: 0.058 - ETA: 0s - loss: 0.056 - ETA: 0s - loss: 0.055 - ETA: 0s - loss: 0.054 - ETA: 0s - loss: 0.054 - ETA: 0s - loss: 0.054 - ETA: 0s - loss: 0.054 - 1s 157ms/step - loss: 0.0549 - val_loss: 0.1907
roc_auc 0.5767444880994902
pr_auc_norm where normal is the positive class 0.1322701862567473
pr_auc_norm where anomaly is the positive class 0.9187449734766535
Class: 1
Epoch 1/10
8/8 [==============================] - ETA: 9s - loss: 0.452 - ETA: 0s - loss: 0.416 - ETA: 0s - loss: 0.392 - ETA: 0s - loss: 0.370 - ETA: 0s - loss: 0.351 - ETA: 0s - loss: 0.333 - ETA: 0s - loss: 0.318 - ETA: 0s - loss: 0.310 - 3s 220ms/step - los

Epoch 5/10
8/8 [==============================] - ETA: 0s - loss: 0.082 - ETA: 0s - loss: 0.084 - ETA: 0s - loss: 0.083 - ETA: 0s - loss: 0.084 - ETA: 0s - loss: 0.083 - ETA: 0s - loss: 0.083 - ETA: 0s - loss: 0.082 - ETA: 0s - loss: 0.081 - 1s 142ms/step - loss: 0.0818 - val_loss: 0.1797
Epoch 6/10
8/8 [==============================] - ETA: 0s - loss: 0.075 - ETA: 0s - loss: 0.077 - ETA: 0s - loss: 0.076 - ETA: 0s - loss: 0.075 - ETA: 0s - loss: 0.075 - ETA: 0s - loss: 0.074 - ETA: 0s - loss: 0.074 - ETA: 0s - loss: 0.074 - 1s 181ms/step - loss: 0.0746 - val_loss: 0.1845
Epoch 7/10
8/8 [==============================] - ETA: 1s - loss: 0.070 - ETA: 0s - loss: 0.069 - ETA: 0s - loss: 0.067 - ETA: 0s - loss: 0.068 - ETA: 0s - loss: 0.068 - ETA: 0s - loss: 0.068 - ETA: 0s - loss: 0.067 - ETA: 0s - loss: 0.068 - 1s 173ms/step - loss: 0.0684 - val_loss: 0.1868
Epoch 8/10
8/8 [==============================] - ETA: 1s - loss: 0.067 - ETA: 1s - loss: 0.066 - ETA: 0s - loss: 0.063 - ETA: 0s 

9/9 [==============================] - ETA: 16s - loss: 0.40 - ETA: 1s - loss: 0.3784 - ETA: 0s - loss: 0.347 - ETA: 0s - loss: 0.322 - ETA: 0s - loss: 0.299 - ETA: 0s - loss: 0.280 - ETA: 0s - loss: 0.267 - ETA: 0s - loss: 0.255 - ETA: 0s - loss: 0.254 - 4s 196ms/step - loss: 0.2543 - val_loss: 0.2127
Epoch 2/10
9/9 [==============================] - ETA: 1s - loss: 0.153 - ETA: 0s - loss: 0.153 - ETA: 0s - loss: 0.147 - ETA: 0s - loss: 0.141 - ETA: 0s - loss: 0.138 - ETA: 0s - loss: 0.134 - ETA: 0s - loss: 0.131 - ETA: 0s - loss: 0.129 - 1s 153ms/step - loss: 0.1291 - val_loss: 0.1858
Epoch 3/10
9/9 [==============================] - ETA: 0s - loss: 0.104 - ETA: 1s - loss: 0.104 - ETA: 0s - loss: 0.104 - ETA: 0s - loss: 0.103 - ETA: 0s - loss: 0.101 - ETA: 0s - loss: 0.100 - ETA: 0s - loss: 0.099 - ETA: 0s - loss: 0.098 - 1s 147ms/step - loss: 0.0982 - val_loss: 0.1728
Epoch 4/10
9/9 [==============================] - ETA: 1s - loss: 0.087 - ETA: 1s - loss: 0.089 - ETA: 0s - loss: 0.

9/9 [==============================] - ETA: 1s - loss: 0.064 - ETA: 1s - loss: 0.066 - ETA: 0s - loss: 0.067 - ETA: 0s - loss: 0.067 - ETA: 0s - loss: 0.068 - ETA: 0s - loss: 0.068 - ETA: 0s - loss: 0.068 - ETA: 0s - loss: 0.069 - 1s 158ms/step - loss: 0.0690 - val_loss: 0.1912
Epoch 8/10
9/9 [==============================] - ETA: 1s - loss: 0.064 - ETA: 1s - loss: 0.066 - ETA: 1s - loss: 0.066 - ETA: 0s - loss: 0.064 - ETA: 0s - loss: 0.064 - ETA: 0s - loss: 0.063 - ETA: 0s - loss: 0.063 - ETA: 0s - loss: 0.062 - 1s 165ms/step - loss: 0.0628 - val_loss: 0.1875
Epoch 9/10
9/9 [==============================] - ETA: 1s - loss: 0.060 - ETA: 1s - loss: 0.061 - ETA: 1s - loss: 0.059 - ETA: 0s - loss: 0.059 - ETA: 0s - loss: 0.060 - ETA: 0s - loss: 0.059 - ETA: 0s - loss: 0.059 - ETA: 0s - loss: 0.058 - 2s 179ms/step - loss: 0.0587 - val_loss: 0.1832
Epoch 10/10
9/9 [==============================] - ETA: 0s - loss: 0.052 - ETA: 0s - loss: 0.052 - ETA: 0s - loss: 0.054 - ETA: 0s - loss: 0.